In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.metrics import mean_squared_error
import numpy as np
from keras import backend as K
import os

In [9]:
# define the directory where the pickle files are stored
folder_path = '../batched_data_pickle_files/'

# define the filenames for the pickle files
file_names = ['processed_train_data.pkl', 'processed_train_targets.pkl', 'processed_test_data.pkl', 'true_rul.pkl']

# loop through each file and load its contents as arrays
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    
    # read the pickle file
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    
    # ensure data is a numpy array, if not convert it
    if isinstance(data, np.ndarray):
        globals()[file_name.replace('.pkl', '')] = data
    else:
        globals()[file_name.replace('.pkl', '')] = np.array(data)

print("Processed Train Data Shape:", processed_train_data.shape)
print("Processed Train Target Shape:", processed_train_targets.shape)
print("Processed Test Data Shape:", processed_test_data.shape)
print("True RUL Shape:", true_rul.shape)

Processed Train Data Shape: (17731, 30, 14)
Processed Train Target Shape: (17731,)
Processed Test Data Shape: (100, 30, 14)
True RUL Shape: (100,)


In [10]:
processed_train_data, processed_val_data, processed_train_targets, processed_val_targets = train_test_split(processed_train_data,
                                                                                                            processed_train_targets,
                                                                                                            test_size = 0.2,
                                                                                                            random_state = 83)
print("Processed train data shape: ", processed_train_data.shape)
print("Processed validation data shape: ", processed_val_data.shape)
print("Processed train targets shape: ", processed_train_targets.shape)
print("Processed validation targets shape: ", processed_val_targets.shape)

Processed train data shape:  (14184, 30, 14)
Processed validation data shape:  (3547, 30, 14)
Processed train targets shape:  (14184,)
Processed validation targets shape:  (3547,)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from sklearn.model_selection import train_test_split
import kerastuner as kt
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.metrics import mean_squared_error
import numpy as np

from keras import backend as K

# After training a model
K.clear_session()


def build_model(hp):
    model = Sequential()

    # Fixed batch size
    batch_size = 128

    # Determine number of LSTM layers
    num_lstm_layers = hp.Int('num_lstm_layers', 1, 3)  # Choose between 1 and 3 LSTM layers

    # Adding LSTM layers based on the chosen number of layers
    for i in range(num_lstm_layers):
        if i == 0:
            # First LSTM Layer
            model.add(layers.LSTM(
                units=hp.Int('units_1', min_value=128, max_value=256, step=32),
                input_shape=(processed_train_data.shape[1], processed_train_data.shape[2]),
                return_sequences=True,
                activation="tanh"
            ))
        else:
            # Subsequent LSTM Layers
            model.add(layers.LSTM(
                units=hp.Int(f'units_{i + 1}', min_value=32, max_value=128, step=32),
                return_sequences=True if i < num_lstm_layers - 1 else False,
                activation="tanh"
            ))
        
        # Remove the dropout and BatchNormalization layers
        # Only LSTM layers remain in this part

    # Determine number of Dense layers
    num_dense_layers = hp.Int('num_dense_layers', 1, 2)  # Choose between 1 and 2 Dense layers

    # Adding Dense layers based on the chosen number of layers
    for j in range(num_dense_layers):
        model.add(layers.Dense(
            units=hp.Int(f'dense_units_{j + 1}', min_value=32, max_value=256, step=32),
            activation="relu"
        ))

    model.add(layers.Dense(1))  # Final output layer

    # Compile Model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)  # No log scaling
        ), 
        loss="mse"
    )
    
    return model

tf.keras.backend.clear_session()

# Set up Keras Tuner with Hyperband
tuner = kt.Hyperband(
    build_model,
    objective="val_loss",
    max_epochs=10,
    factor=3,
    directory="hyperparam_tuning",
    project_name="lstm_tuning_refined"
)

# Run the tuner search
tuner.search(
    processed_train_data, processed_train_targets,
    epochs=10,
    validation_data=(processed_val_data, processed_val_targets),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)],
    verbose=2
)

# Get the best model and evaluate it
best_model = tuner.get_best_models(num_models=1)[0]
val_loss = best_model.evaluate(processed_val_data, processed_val_targets, verbose=0)
print(f"\nBest model validation MSE: {val_loss:.4f}")

# Get the best hyperparameters from the tuner
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:", best_hyperparameters.values)

In [ ]:
Trial 30 Complete [00h 03m 33s] val_loss: 128.57681274414062

Best val_loss So Far: 45.899444580078125 Total elapsed time: 00h 41m 41s

Best model validation MSE: 45.8994 Best hyperparameters: {'num_lstm_layers': 2, 'units_1': 224, 'num_dense_layers': 1, 'dense_units_1': 160, 'learning_rate': 0.0019205962288415628, 'units_2': 32, 'dense_units_2': 192, 'units_3': 96, 'tuner/epochs': 10, 'tuner/initial_epoch': 4, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}

In [35]:
import numpy as np
import tensorflow as tf
from keras import layers, Sequential
from keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.metrics import mean_squared_error

tf.keras.backend.clear_session()

# Best hyperparameters
best_hyperparameters = {
    'num_lstm_layers': 2,         # Two LSTM layers
    'units_1': 224,               # First LSTM layer units
    'num_dense_layers': 1,        # One Dense layer
    'dense_units_1': 160,         # Dense layer units
    'learning_rate': 0.001,       # 0.0019205962288415628
    'units_2': 32,                # Second LSTM layer units
}

# Define the function to build the LSTM model with the best hyperparameters
def build_best_model(hyperparams, input_shape):
    model = Sequential()

    # First LSTM Layer
    model.add(layers.LSTM(
        units=hyperparams['units_1'],
        input_shape=input_shape,
        return_sequences=True
    ))

    # Second LSTM Layer (last layer, don't return sequences)
    model.add(layers.LSTM(units=hyperparams['units_2']))

    # Dense Layer
    model.add(layers.Dense(hyperparams['dense_units_1'], activation='relu'))

    # Output Layer
    model.add(layers.Dense(1))  # Assuming a regression task

    # Compile Model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hyperparams['learning_rate']),
        loss='mse'
    )

    return model

# Learning rate scheduler function
def lr_scheduler(epoch):
    if epoch < 5:
        return best_hyperparameters['learning_rate']  # Use initial learning rate
    else:
        return 0.0001  # Set to 0.0001 after epoch 5

# Example input shape (timesteps, features), modify based on your data
input_shape = (processed_train_data.shape[1], processed_train_data.shape[2])  # Define your input shape

# Build the model
best_model = build_best_model(best_hyperparameters, input_shape)

# Set up early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Stop training after 3 epochs of no improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

# Set up learning rate scheduler
lr_scheduler_callback = LearningRateScheduler(lr_scheduler, verbose=1)

# Train the model
history = best_model.fit(
    processed_train_data, 
    processed_train_targets,
    epochs=10,  # Set your desired number of epochs (tuner/epochs)
    validation_data=(processed_val_data, processed_val_targets),
    callbacks=[early_stopping, lr_scheduler_callback],
    verbose=2
)

# Make predictions (replace test_data with your actual test dataset)
predictions = best_model.predict(processed_test_data)



Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/10
444/444 - 22s - 49ms/step - loss: 2450.4880 - val_loss: 724.2559 - learning_rate: 1.0000e-03

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/10
444/444 - 19s - 42ms/step - loss: 474.3666 - val_loss: 382.4680 - learning_rate: 1.0000e-03

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/10
444/444 - 19s - 42ms/step - loss: 247.2537 - val_loss: 207.7093 - learning_rate: 1.0000e-03

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/10
444/444 - 21s - 46ms/step - loss: 198.3183 - val_loss: 163.2923 - learning_rate: 1.0000e-03

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/10
444/444 - 19s - 43ms/step - loss: 185.5225 - val_loss: 193.0704 - learning_rate: 1.0000e-03

Epoch 6: LearningRateScheduler setting learning rate to 0.0001.
Epoch 6/10
444/444 - 20s - 46ms/step - loss: 160.5922 - val_loss: 150.9580 - learning_rate: 1.0000e

In [36]:
len(predictions)

100

In [37]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def evaluate_rul_metrics(true, predicted):
    
    true = np.array(true)
    predicted = np.array(predicted)
    
    mae = float(mean_absolute_error(true, predicted))
    mse = float(mean_squared_error(true, predicted))
    rmse = float(np.sqrt(mse))
    mape = float(np.mean(np.abs((true - predicted) / true)) * 100)
    
    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE": mape
    }
    
metrics = evaluate_rul_metrics(true_rul, predictions)
metrics


{'MAE': 10.672525138854981,
 'MSE': 212.84601973255033,
 'RMSE': 14.589243288551684,
 'MAPE': 148.36926763230426}

In [30]:
import os
import json

# Path to your tuning logs directory
log_dir = '../ml_models/hyperparam_tuning/lstm_tuning_refined/'

# Initialize variables to keep track of the best trial and its score
best_trial = None
best_score = float('inf')  # Assuming lower score is better, adjust if higher is better

# Walk through the log directory and find all the trial folders
for trial_folder in os.listdir(log_dir):
    trial_folder_path = os.path.join(log_dir, trial_folder)
    
    # Check if this is a folder (trial folder)
    if os.path.isdir(trial_folder_path):
        trial_json_path = os.path.join(trial_folder_path, 'trial.json')
        
        # Check if the trial.json file exists in this folder
        if os.path.isfile(trial_json_path):
            # Read the JSON file
            with open(trial_json_path, 'r') as f:
                trial_data = json.load(f)
                
                # Extract the performance score (e.g., validation loss or other metric)
                score = trial_data.get('score')
                
                # Check if this is the best trial based on the score
                if score is not None and score < best_score:
                    best_score = score
                    best_trial = trial_data

# After the loop, best_trial contains the best trial's hyperparameters
if best_trial is not None:
    print("Best Trial Contents:")
    print(json.dumps(best_trial, indent=2))
else:
    print("No trials found.")



Best Trial Contents:
{
  "trial_id": "0016",
  "hyperparameters": {
    "space": [
      {
        "class_name": "Int",
        "config": {
          "name": "num_lstm_layers",
          "default": null,
          "conditions": [],
          "min_value": 1,
          "max_value": 3,
          "step": 1,
          "sampling": "linear"
        }
      },
      {
        "class_name": "Int",
        "config": {
          "name": "units_1",
          "default": null,
          "conditions": [],
          "min_value": 128,
          "max_value": 256,
          "step": 32,
          "sampling": "linear"
        }
      },
      {
        "class_name": "Int",
        "config": {
          "name": "num_dense_layers",
          "default": null,
          "conditions": [],
          "min_value": 1,
          "max_value": 2,
          "step": 1,
          "sampling": "linear"
        }
      },
      {
        "class_name": "Int",
        "config": {
          "name": "dense_units_1",
          "